In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import time

from gpuNUFFT import NUFFTOp

# Using NUFFTOp directly

Samples should be in [-0.5, 0.5[

## 2D multi-coil tests

### Preparing input data

In [ ]:
res = [64, 128]
[x, y] = np.meshgrid(np.linspace(-1, 1, res[0]),
                        np.linspace(-1, 1, res[1]))
img = (x**2 + y**2  < 0.5**2).T
img = img.astype(np.complex64)
print('Input image shape is', img.shape)
sli = 18
plt.figure(1)
plt.imshow(abs(img[...]), cmap='gray')
plt.title('image')

n_samples = np.prod(res)
kcoords = (np.random.uniform(size=(n_samples, 2)) -0.5).astype(np.float32)
print('Input kcoords shape is', kcoords.shape)

plt.figure(2)
plt.plot(kcoords[:,0], kcoords[:,1], '.')
plt.xlim([-0.6,0.6])
plt.ylim([-0.6,0.6])
plt.axis('square')
plt.title('samples')

weights = np.ones(kcoords.shape[0])
print('Input weights shape is', weights.shape)

In [ ]:
# Coil maps shape should be n_coils x res
n_coils = 2
coil_maps = np.ones([n_coils, *np.array(res).T], dtype=np.complex64)
print('Input coil maps shape is', coil_maps.shape)

### Applying nufft

In [ ]:
wg = 3
sw = 8
osf = 2
n_coils = 1
balance_workload = True
A = NUFFTOp(
    np.reshape(kcoords, kcoords.shape[::-1], order='F'),
    res,
    coil_maps,
    weights,
    wg,
    sw,
    osf,
    balance_workload,
)
x = A.op(np.reshape(img.T, img.size))
print('Output kdata shape is', x.shape)
img_adj = A.adj_op(x.reshape(-1)).T
print('Output adjoint img shape is', img_adj.shape)
img_adj = np.squeeze(img_adj)
print(img_adj.shape)

sli = 18
plt.figure(3)
plt.imshow(abs(img_adj), cmap='gray')
plt.title('adjoint image')

del A

## 3D multi-coil tests

### Preparing input data

In [ ]:
res = [64, 64, 32]
[x, y, z] = np.meshgrid(np.linspace(-1, 1, res[0]),
                        np.linspace(-1, 1, res[1]),
                        np.linspace(-1, 1, res[2]))
img = (x**2 + y**2 + z**2 < 0.5**2).transpose([1, 0, 2])
print('Input image shape is', img.shape)

sli = img.shape[-1] // 2
plt.figure(1)
plt.imshow(img[..., sli], cmap='gray'),
plt.title('image')

n_samples = np.prod(res) // 4
kcoords =  (np.random.uniform(size=(n_samples, 3)) -0.5)
print('Input samples shape is', kcoords.shape)

plt.figure(2)
plt.plot(kcoords[:,1], kcoords[:,2], '.')
plt.xlim([-0.6, 0.6])
plt.ylim([-0.6, 0.6])
plt.axis('square')
plt.title('samples')

weights = np.ones(kcoords.shape[0])
print('Input weights shape is', weights.shape)

In [ ]:
# Coil maps shape should be n_coils x res
n_coils = 8
coil_maps = np.ones([n_coils, *np.array(res).T], dtype=np.complex64)
print('Input coil maps shape is', coil_maps.shape)

### Applying multi-coil nufft

In [ ]:
wg = 3
sw = 8
osf = 2
time0 = time.time()
n_coils = 1
A = NUFFTOp(
    np.reshape(kcoords, kcoords.shape[::-1], order='F'),
    res,
    coil_maps,
    weights,
    wg,
    sw,
    osf,
    True,
)
print('time of NUFFT instantiation = {} seconds'.format(time.time() - time0))

time1 = time.time()
x = A.op(np.reshape(img.T, img.size))
print('time of forward op = {} seconds'.format(time.time() - time1))

print('Output kdata shape is', x.shape)

time2 = time.time()
img_back = np.squeeze(A.adj_op(x).T)
print('time of adj op = {}'.format(time.time() - time2))

print('Output adjoint image shape is', img_back.shape)

del A

fig, axs = plt.subplots(nrows=1, ncols=2)
axs[0].imshow(abs(img_back[..., res[2] // 2]), aspect='equal', cmap='gray')
axs[1].imshow(abs(img_back[res[0] // 2, ...]), aspect='equal', cmap='gray')